In [ ]:
import pytorch_lightning as pl
from data.profile_dataset import DS, DataModuleClass
import torch
from matplotlib.pyplot import cycler
from matplotlib.colors import Normalize
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from experiments import DIVA_EXP
from models import DIVA_v1, DIVA_v2
from pathlib import Path
import os
import numpy as np
SMALL_SIZE = 20
MEDIUM_SIZE = 22
BIGGER_SIZE = 24

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

def de_standardize(x, mu, var):
    return (x*var) + mu


In [ ]:
pin_memory=False
cpus_per_trial=4
data_dir = "/home/local/kitadam/ENR_Sven/moxie/data/processed/profile_database_v1_psi22.hdf5"
num_epochs=1
gpus_per_trial=0

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

generator=torch.Generator().manual_seed(42)
torch.manual_seed(42)
logger = TensorBoardLogger("tb_logs", name='ModelVis')

STATIC_PARAMS = {'data_dir':data_dir,
                'num_workers': cpus_per_trial,
                'pin_memory': pin_memory}

HYPERPARAMS = {'LR': 0.001, 'weight_decay': 0.0, 'batch_size': 512}

model_hyperparams = {'in_ch': 2, 'out_dim':63,
                        'mach_latent_dim': 40, 'beta_stoch': 0.01, 'beta_mach':  100.,
                        'alpha_mach': 1.0, 'alpha_prof': 1.0,
                    'loss_type': 'semi-supervised'}


params = {**STATIC_PARAMS, **HYPERPARAMS, **model_hyperparams}
model = DIVA_v2(**model_hyperparams)

trainer_params = {'max_epochs': num_epochs,  'gpus': gpus_per_trial if str(device).startswith('cuda') else 0,
                'gradient_clip_val': 0.5, 'gradient_clip_algorithm':"value",
                'profiler':"simple"}


experiment = DIVA_EXP(model, params)
runner = pl.Trainer(logger=logger, **trainer_params)


datacls = DataModuleClass(**params)

runner.fit(experiment, datamodule=datacls)
runner.test(experiment, datamodule=datacls)

In [ ]:

LABEL = ['Q95', 'RGEO', 'CR0', 'VOLM', 'TRIU', 'TRIL', 'XIP', 'ELON', 'POHM', 'BT', 'ELER', 'P_NBI', 'P_ICRH']

param = 'XIP'
param_idx = LABEL.index(param)
dim_idx = 7
resolution = 100
data_point_index = 100

# Get the full training dataset and normalizing constant for data
train_dataset = datacls.train_set
train_prof, train_mp = train_dataset.X, train_dataset.y


mu_T, var_T = datacls.get_temperature_norms()
mu_MP, var_MP = datacls.get_mp_norms()

# From training dataset, grab profiles
real_density_profiles, real_temperature_profiles = train_prof[:, 0], train_prof[:, 1] 
real_temperature_profiles = de_standardize(real_temperature_profiles, mu_T, var_T)

# And machine params
real_mp_vals = train_mp[:, :]
real_mp_vals = de_standardize(real_mp_vals, mu_MP, var_MP)
real_vals = real_mp_vals[:, param_idx]


In [ ]:
all_subset = real_density_profiles[torch.logical_and(torch.logical_and(real_vals, real_vals > -1671018 - 1), torch.logical_and(real_vals, real_vals < -1671018 + 1))]
mean_sample = torch.mean(real_density_profiles[torch.logical_and(torch.logical_and(real_vals, real_vals > -1671018 - 1), torch.logical_and(real_vals, real_vals < -1671018 + 1))], 0)

torch.mean(100*torch.abs(-all_subset + mean_sample) / mean_sample)




# torch.mean(torch.abs( - real_density_profiles[torch.logical_and(torch.logical_and(real_vals, real_vals > -1671018 - 1), torch.logical_and(real_vals, real_vals < -1671018 + 1))]) / (torch.mean(real_density_profiles[torch.logical_and(torch.logical_and(real_vals, real_vals > -1671018 - 1), torch.logical_and(real_vals, real_vals < -1671018 + 1))], 0)) * 100, 0)

In [ ]:
print(real_density_profiles.shape)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(18, 8), dpi=310, constrained_layout=True)

# label_dict = {key: val.detach().numpy() for key, val in zip(LABEL, real_vals)}

axs[0].set(title='Density')
axs[1].set(title='Temperature')





cmap = plt.cm.get_cmap('coolwarm_r')

# p = 1
last_mp = real_vals[0]

chosen_vals = [last_mp]
list_ps = []
for p in range(0, len(real_vals) ): # 
    if ((real_vals[p]  < last_mp+ 10) & (real_vals[p]  > last_mp - 10)):
        continue 
    else: 
        chosen_vals.append(real_vals[p])
        list_ps.append(p)
        last_mp = real_vals[p]

print(len(list_ps))
norm = Normalize(vmin= min(chosen_vals), vmax=max(chosen_vals))
colors = norm(chosen_vals)

colors = cmap(colors)



axs[0].set_prop_cycle(color=colors)
axs[1].set_prop_cycle(color=colors)


for p in list_ps: 
    if real_vals[p] < -3e6: 
        axs[0].plot(real_density_profiles[p], lw=1.5, alpha=0.8)
        axs[1].plot(real_temperature_profiles[p], lw=1.5, alpha=0.8)
    else:
        axs[0].plot(real_density_profiles[p], lw=1.5, alpha=0.5)
        axs[1].plot(real_temperature_profiles[p], lw=1.5, alpha=0.5)


axs[0].set_xlabel('Radius from core [arb.]')
axs[1].set_xlabel('Radius from core [arb.]')
axs[0].set_ylabel('$n_e \; \; [10^{20}$ m$^{-3}]$', size='xx-large')
axs[1].set_ylabel('$T_e \; \;$ [eV]', size='xx-large')

"""
axs[0].plot(real_density_profile, c='black',label='Real', linestyle='--', lw=2, alpha=0.8)
axs[1].plot(real_temperature_profile, c='black', label='\n'.join([f'{key}: {value:.4}' for key, value in label_dict.items()]), linestyle='--', lw=2, alpha=0.8)

axs[0].plot(sample_density_profile, c='forestgreen', label='Generated',linestyle='--', lw=2)
axs[1].plot(sample_temperature_profile, c='forestgreen', linestyle='--', lw=2, alpha=0.8)
"""
axs[0].legend(fontsize=10)
axs[1].legend(fontsize=10)

axs[1].set_ylim(0, 10000)
mappable = plt.cm.ScalarMappable(cmap=cmap)
mappable.set_array(chosen_vals)

cb = fig.colorbar(mappable, shrink=0.7, aspect=5, label='Real $I_P$ [A]',  pad=0.01)


# cb.ax.plot([min(real_val), max(real_val)], [real_val]*2, 'black')# [norm(real_val)]*2)
# cb.ax.plot([min(real_val), max(real_val)], [sample_mp_val]*2, 'forestgreen')# [norm(real_val)]*2)

fig.suptitle('Sweeping the Current: Experimental Data')
plt.show()

In [ ]:
from matplotlib.colors import Normalize
from matplotlib import colors as mcolors
from matplotlib import rcParams
rcParams['axes.labelpad'] = 20

fig = plt.figure(figsize=(25, 12), dpi=400)

# Density Plot
ax = fig.add_subplot(1, 2, 1, projection='3d')

x = np.arange(0, 63)# np.repeat([np.arange(0, 63), ], resolution, 0)
y = chosen_vals

X, Y = np.meshgrid(x, y)

chosen_profiles = np.zeros((2186, 63))
i = 0
for p in list_ps: 
    chosen_profiles[i] = (real_density_profiles[p].detach().numpy())
    
Z = np.array(chosen_profiles)

print(X.shape, Y.shape, Z.shape)
norm = Normalize()
colors = norm(Y)

cmap = plt.cm.get_cmap("coolwarm_r")

surface = ax.plot_surface(X, Y, Z, facecolors=cmap(colors), alpha=0.75)
# ax.set_zlim(0, 0.5)


mappable = plt.cm.ScalarMappable(cmap=cmap)
mappable.set_array(Y)
cp = fig.colorbar(mappable, shrink=0.5, aspect=5, label='$I_P$ [A]', orientation='horizontal',  pad=0.01)



ax.set(xlabel='R [arb.]', ylabel='$I_P$ [A]')
ax.zaxis.set_rotate_label(False)  # disable automatic rotation
ax.set_zlabel('$n_e \; \; [10^{20}$ m$^{-3}]$', rotation=90)
# ax.view_init(30,-115)
plt.show()